In [1]:
import numpy as np
import utils
import qiskit
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
provider = qiskit.IBMQ.load_account()
simulator = qiskit.Aer.get_backend('qasm_simulator')

## Testing metrics

In [10]:
lima5_result = utils.result_from_csv('measured_data/linear5_lima.csv')

In [13]:
linear_interactions = {(i, i + 1): 1 for i in range(4)}
linear_circ = utils.linear_circuit(5, linear_interactions)
linear_circ.measure_all()
job_sim = qiskit.execute(linear_circ, simulator, shots=8192)
sim_result = job_sim.result().get_counts()

Each function returns its metric and the boolean flag, which is True if result contains all possible observations and False otherwise

In [19]:
utils.kullback_leibler_divergence(sim_result, linear_interactions),\
utils.kullback_leibler_divergence(lima5_result, linear_interactions)

((0.002199073889893586, False), (0.3345491018909429, True))

In [24]:
utils.hellinger_distance(sim_result, linear_interactions),\
utils.hellinger_distance(lima5_result, linear_interactions)

((0.023498918269350376, False), (0.26936296772384316, True))

In [26]:
utils.total_variation_distance(sim_result, linear_interactions),\
utils.total_variation_distance(lima5_result, linear_interactions)

((0.016322934274533046, False), (0.29839518663330333, True))

## Measuring on real IBM data

In [3]:
circuits = []
for n in range(1, 16):
    linear_interactions = {(i, i + 1): 1 for i in range(n - 1)}
    linear_circ = utils.linear_circuit(i, linear_interactions)
    linear_circ.measure_all()
    circuits.append(linear_circ)

In [10]:
backend = provider.get_backend('ibmq_16_melbourne')
job = qiskit.execute(circuits, backend, shots=8192)
qiskit.tools.job_monitor(job)

Job Status: job has successfully run


KeyboardInterrupt: 

In [11]:
results = []
for i in range(1, 16):
    results.append(utils.result_from_csv(f'measured_data/melbourne_linear/{i}.csv'))

In [12]:
measures = []
for n, result in enumerate(results, 1):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    probability_denomenator = utils.Z(n, interactions, beta=1)
    
    job_sim = qiskit.execute(circuits[n - 1], simulator, shots=8192)
    sim_result = job_sim.result().get_counts()
    
    ibm_kl, all_obtained = utils.kullback_leibler_divergence(result, interactions, probability_denomenator)
    sim_kl, sim_all_obtained = utils.kullback_leibler_divergence(sim_result, interactions, probability_denomenator)
    
    ibm_l2, _ = utils.hellinger_distance(result, interactions, probability_denomenator)
    sim_l2, _ = utils.hellinger_distance(sim_result, interactions, probability_denomenator)
    
    ibm_tv, _ = utils.total_variation_distance(result, interactions, probability_denomenator)
    sim_tv, _ = utils.total_variation_distance(sim_result, interactions, probability_denomenator)
    
    measures.append([n, ibm_kl, sim_kl, ibm_l2, sim_l2, ibm_tv, sim_tv, all_obtained, sim_all_obtained])

In [13]:
df = pd.DataFrame(measures, columns=['n', 'IBM KL', 'SIM KL', 'IBM L2', 'SIM L2', 'IBM TV', 'SIM TV', 'All obtained', 'Sim all obtained'])
df

,n,IBM KL,SIM KL,IBM L2,SIM L2,IBM TV,SIM TV,All obtained,Sim all obtained
0,1,0.005342,0.000142,0.036573,0.005956,0.051636,0.008423,True,True
1,2,0.081075,0.000020,0.134827,0.002244,0.146788,0.002044,True,True
2,3,0.226552,0.000403,0.212241,0.009943,0.234910,0.008103,True,True
3,4,0.437194,0.000885,0.287081,0.015071,0.282055,0.010875,True,True
4,5,0.619743,0.002543,0.345037,0.024799,0.371669,0.015254,True,False
5,6,0.918285,0.003492,0.408567,0.027007,0.441235,0.017956,True,False
6,7,1.194131,0.007374,0.463867,0.038307,0.511753,0.030331,True,False
7,8,1.300959,0.016422,0.495115,0.051682,0.537188,0.038681,True,False
8,9,1.609797,0.021848,0.542326,0.058442,0.586338,0.042686,False,False
9,10,1.829783,0.041829,0.563799,0.077273,0.613012,0.060508,False,False


## Estemating D-Wave beta

In [2]:
import dimod
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite

In [2]:
from scipy.optimize import minimize
from tqdm import tqdm

In [3]:
sampler = DWaveSampler() 
embedding_sampler = EmbeddingComposite(sampler)

### Using obtained distributions

In [32]:
opt_betas = []
for n in tqdm(range(1, 16)):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    result = utils.result_from_csv(f'measured_data/dwave_linear/{n}.csv')
    
    res_kl = minimize(lambda beta: utils.kullback_leibler_divergence(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_kl, val_kl = res_kl.x[0], res_kl.fun
    
    res_l2 = minimize(lambda beta: utils.hellinger_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_l2, val_l2 = res_l2.x[0], res_l2.fun
    
    res_tv = minimize(lambda beta: utils.total_variation_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_tv, val_tv = res_tv.x[0], res_tv.fun
    
    opt_betas.append([n, beta_kl, val_kl, beta_l2, val_l2, beta_tv, val_tv])

100%|██████████| 15/15 [02:00<00:00,  8.03s/it]


In [11]:
hard_opt_betas = []
for n in tqdm([50, 100, 200, 500, 1000, 1500, 2000]):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    result = utils.result_from_csv(f'measured_data/dwave_linear/{n}.csv')
    
    res_kl = minimize(lambda beta: utils.kullback_leibler_divergence(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_kl, val_kl = res_kl.x[0], res_kl.fun
    
    res_l2 = minimize(lambda beta: utils.hellinger_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_l2, val_l2 = res_l2.x[0], res_l2.fun
    
    res_tv = minimize(lambda beta: utils.total_variation_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_tv, val_tv = res_tv.x[0], res_tv.fun
    
    hard_opt_betas.append([n, beta_kl, val_kl, beta_l2, val_l2, beta_tv, val_tv])

 57%|█████▋    | 4/7 [04:50<05:08, 102.72s/it]/home/user/quantum_computer/utils.py:37: RuntimeWarning: overflow encountered in exp
  return np.exp(-beta * H(spins, interactions, magnetic_fields))
/home/user/quantum_computer/utils.py:324: RuntimeWarning: invalid value encountered in double_scalars
  Q = energy_map[outcome] / probability_denomenator
/home/user/quantum_computer/utils.py:351: RuntimeWarning: invalid value encountered in double_scalars
  Q = energy_map[outcome] / probability_denomenator
/home/user/quantum_computer/utils.py:378: RuntimeWarning: invalid value encountered in double_scalars
  Q = energy_map[outcome] / probability_denomenator
100%|██████████| 7/7 [28:26<00:00, 243.73s/it]


In [12]:
df = pd.DataFrame(hard_opt_betas, columns=['n', 'Beta KL', 'Metric KL', 'Beta L2', 'Metric L2', 'Beta TV', 'Metric TV'])
df

,n,Beta KL,Metric KL,Beta L2,Metric L2,Beta TV,Metric TV
0,50,2.252617,0.101288,2.350700,0.147163,2.488517,0.118886
1,100,1.518808,0.742410,1.581756,0.421983,2.363651,0.459613
2,200,0.139812,0.231503,0.078868,0.194882,0.047964,0.154379
3,500,0.000073,0.000047,0.000061,0.003234,0.000006,0.000122
4,1000,1.000000,NaN,1.000000,NaN,1.000000,0.000000
5,1500,1.000000,NaN,1.000000,NaN,1.000000,0.000000
6,2000,1.000000,NaN,1.000000,NaN,1.000000,0.000000


### Using energy distributions

In [24]:
opt_betas = []
for n in tqdm(list(range(1, 16)) + [50, 100, 200, 500, 1000, 1500, 2000]):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    result = utils.result_from_csv(f'measured_data/dwave_linear/{n}.csv')
    
    res_kl = minimize(lambda beta: utils.kullback_leibler_divergence_energy(result, beta=beta[0]), 1, bounds=[(0.0, 100.0)])
    beta_kl, val_kl = res_kl.x[0], res_kl.fun
    
    res_l2 = minimize(lambda beta: utils.hellinger_distance_energy(result, beta=beta[0]), 1, bounds=[(0.0, 100.0)])
    beta_l2, val_l2 = res_l2.x[0], res_l2.fun
    
    opt_betas.append([n, beta_kl, val_kl, beta_l2, val_l2])

 50%|█████     | 11/22 [00:00<00:00, 103.21it/s]/home/user/quantum_computer/utils.py:390: RuntimeWarning: overflow encountered in exp
  return 2 * comb(n - 1, alternations_n) * exp(-beta * cur_H)
/home/user/quantum_computer/utils.py:390: RuntimeWarning: overflow encountered in multiply
  return 2 * comb(n - 1, alternations_n) * exp(-beta * cur_H)
/home/user/quantum_computer/utils.py:408: RuntimeWarning: invalid value encountered in double_scalars
  P = empiric_frequency
 50%|█████     | 11/22 [00:16<00:00, 103.21it/s]/home/user/quantum_computer/utils.py:425: RuntimeWarning: overflow encountered in exp
  cur_H = trivial_H(n, alternations_n)
/home/user/quantum_computer/utils.py:425: RuntimeWarning: overflow encountered in multiply
  cur_H = trivial_H(n, alternations_n)
/home/user/quantum_computer/utils.py:443: RuntimeWarning: invalid value encountered in double_scalars
  for alternations_n, empiric_frequency in empiric_energy_prob.items():
 91%|█████████ | 20/22 [01:06<00:06,  3.30s/it] 

OverflowError: int too large to convert to float

In [23]:
df = pd.DataFrame(opt_betas, columns=['n', 'Beta KL', 'Metric KL', 'Beta L2', 'Metric L2'])
df

,n,Beta KL,Metric KL,Beta L2,Metric L2
0,1,1.000000,0.000000e+00,1.000000,0.000000e+00
1,2,3.955563,1.195663e-10,3.955967,6.764585e-11
2,3,3.953391,1.438955e-07,3.955784,4.745071e-08
3,4,4.155419,1.952625e-07,4.158517,6.331157e-08
4,5,4.099527,4.530657e-07,4.099442,1.602434e-07
5,6,3.864663,1.932940e-06,3.863890,6.845830e-07
6,7,3.519846,5.359293e-04,3.550772,8.834512e-03
7,8,3.714606,6.463486e-04,3.758444,9.320332e-03
8,9,3.559601,4.239316e-04,3.580674,8.198567e-03
9,10,3.239601,4.816067e-04,3.252405,9.431835e-03
